In [64]:
import PyPDF2
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from dateutil.parser import parse
import os
import pandas as pd
import numpy as np
import ast 

In [15]:
def convert_pdf_to_string(file_path):

	output_string = StringIO()
	with open(file_path, 'rb') as in_file:
	    parser = PDFParser(in_file)
	    doc = PDFDocument(parser)
	    rsrcmgr = PDFResourceManager()
	    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
	    interpreter = PDFPageInterpreter(rsrcmgr, device)
	    for page in PDFPage.create_pages(doc):
	        interpreter.process_page(page)

	return(output_string.getvalue())

                
def convert_title_to_filename(title):
    filename = title.lower()
    filename = filename.replace(' ', '_')
    return filename

def find_title_page(table_of_contents_raw):
    lll=[]
    for i,item in enumerate(table_of_contents_raw):
        item=item.strip()
        try:
            tr = is_date(item, fuzzy=False)
        except:
            tr = False
        if((not item.isdecimal())&(not tr)):
            a,b=0,0
            split = item.split()
            for kk in split:
                if(type(kk)==str):
                    a=1
                if(kk.isdecimal()):
                    b=1
            if(a+b==2):
                lll.append(item)
    return lll


def split_to_title_and_pagenum(table_of_contents_entry):
    title_and_pagenum = table_of_contents_entry.strip()
    title_and_pagenum = title_and_pagenum.lower()
    title_and_pagenum = title_and_pagenum.replace('article', '')
    title_and_pagenum = title_and_pagenum.replace('schedule','')
    title_and_pagenum = title_and_pagenum.replace('section','')
    title_and_pagenum = title_and_pagenum.replace('annex','')
    title_and_pagenum = title_and_pagenum.strip()
    if len(title_and_pagenum) > 0:
        try:
            if title_and_pagenum[1].isdigit():
                j = 1
                try:
                    while title_and_pagenum[j].isdigit():
                        j += 1
                except:
                    pass
                title_and_pagenum = title_and_pagenum.replace(title_and_pagenum[:j],'')
                title_and_pagenum = title_and_pagenum.strip()
        except:
            pass
    title = None
    pagenum = None
    
    if len(title_and_pagenum) > 0:
        if title_and_pagenum[-1].isdigit():
            i = -1
            try:
                while title_and_pagenum[i].isdigit():
                    i -= 1
            except:
                pass
            i = i+1
            title = title_and_pagenum[:i].strip()
            pagenum = title_and_pagenum[i:].strip()
            if (('i'or'v' or 'x') in title[:4] and ' ' in title[:5]):
                title = title[:4].replace('i','')+title[4:]
                title = title[:4].replace('v','')+title[4:]
                title = title[:4].replace('x','')+title[4:]
                title = title.strip()
            if(('i'or'v' or 'x') in title[:3] and ' ' in title[:4]):
                title = title[:3].replace('i','')+title[3:]
                title = title[:3].replace('v','')+title[3:]
                title = title[:3].replace('x','')+title[3:]
                title = title.strip()
            if(('i'or'v' or 'x') in title[:2] and ' ' in title[:3]):
                title = title[:2].replace('i','')+title[2:]
                title = title[:2].replace('v','')+title[2:]
                title = title[:2].replace('x','')+title[2:]
                title = title.strip()
            if(('i'or'v' or 'x') in title[:1] and ' ' in title[:2]):
                title = title[:1].replace('i','')+title[1:]
                title = title[:1].replace('v','')+title[1:]
                title = title[:1].replace('x','')+title[1:]
                title = title.strip()
    return title, pagenum

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False
def begend(file):
    begining=1
    ending=9
    for i in range(10):
        if(('contents' or 'page' or 'clause') in file.getPage(i).extractText().lower()): 
            begining = i
            break
    for i in range(begining,10):
        if(('.....') in reader.getPage(i).extractText().lower()): 
            ending = i
        if(not ('.....') in reader.getPage(i).extractText().lower()):
            break
    return begining, ending

def contents_processing(reader):
    begin, finish = begend(reader)
    writer = PyPDF2.PdfFileWriter()
    for i in range(begin, finish+1):  
        writer.addPage(reader.getPage(i))

    output_filename = 'table_of_contents.pdf'

    with open(output_filename, 'wb') as output:
        writer.write(output)

    text = convert_pdf_to_string('table_of_contents.pdf')
    text = text.replace('.','')
    text = text.replace('\x0c','')
    table_of_contents_raw = text.split('\n')
    list_title_pages = find_title_page(table_of_contents_raw)
    dictt ={}
    for items in list_title_pages:
        t,p = split_to_title_and_pagenum(items)
        try:
            p=int(p)
        except:
            pass
        dictt.update({t:p})
    return dictt,finish

In [3]:
directory = 'LPAs'
pdf_names = []
contents = []
pagenumendcontents=[]
for i,file in enumerate(os.listdir(directory)):
    derctorys = directory+'/'+file
    print(derctorys)
    pdfFileObj = open(derctorys, 'rb')
    reader = PyPDF2.PdfFileReader(pdfFileObj,strict = False)
    if reader.isEncrypted:
        reader.decrypt("")
    arial,fin = contents_processing(reader)
    fin = fin+1
    pdf_names.append(derctorys)
    contents.append(arial)
    pagenumendcontents.append(fin)

LPAs/(i) CVC Capital Partners VIII LPA _ Initial Investor Draft_fr Albert Pond on 30.03.2020 at 9.36 am(main_28576848_1).PDF
LPAs/01. Dyal Capital Partners V (A) LP - A_R LPA (August 2020).pdf
LPAs/01. Dyal Capital Partners V (A) LP - A_R LPA (August 2020)_fr Albert Pond on 08.10.2020 at 12.03 pm.pdf
LPAs/02. Dyal Capital Partners V (B) LP - A_R LPA (August 2020).pdf
LPAs/02. Dyal Capital Partners V (B) LP - A_R LPA (August 2020)_fr Albert Pond on 08.10.2020 at 12.03 pm.pdf
LPAs/06. HFCP X (Parallel), L.P. - A_R LPA (Revised LPA Feb 2021).pdf
LPAs/1.3 Equistone Fund VI - Draft LPA (Investor Draft #1).pdf
LPAs/1.5.2 IEF V SCSp - LPA  - Draft 210427.pdf
LPAs/11.1.1 iCON V - LPA (Draft 13 December 2019)_fr Albert Pond on 22.01.2020 at 8.59 am(main_27920908_1).PDF
LPAs/11.1.3 iCON V-B - LPA (Draft 13 December 2019)_fr Albert Pond on 21.01.2020 at 4.05 pm(main_27920939_1).PDF
LPAs/11.2 A_R Agreement of Limited Partnership - GEI Side VII.PDF
LPAs/11.6.2 CUBE II Feeder LPA - 07 02 2016_ resta

LPAs/Hillhouse Focused Growth Fund V Feeder L.P. - AR LPA (Executed) (May 1, 2020)_fr Albert Pond on 19.8.2020 at 5.16 pm(main_30339107_1).PDF
LPAs/Hillhouse Focused Growth Fund V, L.P. - AR LPA (Executed) (May 1, 2020)_fr Albert Pond on 20.08.2020 at 1.02 pm(main_30339181_1).PDF
LPAs/Hillhouse Venture Fund V Feeder, L.P. - AR LPA (Executed) (May 1, 2020)_fr Albert Pond on 19.08.2020 at 5.17 pm(main_30339153_1).PDF
LPAs/Hillhouse Venture Fund V, L.P. - AR LPA (Executed) (May 1, 2020)_fr Albert Pond on 20.08.2020 at 1.02 pm(main_30339189_1).PDF
LPAs/HOPU Magnolia - LPA (Draft) vF.pdf
LPAs/KKR Asian Fund IV SCSp - AR LPA.PDF
LPAs/KPS Special Situations Fund V (A), LP - A_R LPA (Distribution Version).pdf
LPAs/KPS Special Situations Mid-Cap Fund (A), LP - A_R LPA (Distribution Vers....pdf
LPAs/Landmark Equity Partners XVII Offshore L.P. - A_R LPA (K_E Draft).PDF


LPAs/Linden Capital Partners V[-A] - LPA (Execution Copy).pdf
LPAs/LPA - CB Offshore Equity Fund X.pdf
LPAs/LPA - Charlesbank Equity Fund X.pdf
LPAs/LPA - CVC Strategic Opportunities.pdf
LPAs/LPA - Seventh Cinven Fund Feeder LP Investor Draft 2.pdf
LPAs/LPA - Seventh Cinven Fund No.1 LP Investor (Master).pdf
LPAs/LPA.pdf
LPAs/LPA1.pdf
LPAs/LPA_Carlyle Global Partners II, L.P._dated January 25, 2019_fr Albert Pond on 25.06.19 at 4.46 pm(main_25451580_1).PDF
LPAs/LPA_Hollyport Secondary Opportunities VII LP - 2nd A_R_fr Albert Pond on 26.09.19 at 10.03 am(main_26579024_1).PDF
LPAs/LPA_KKR Core Investments Partnership SCSp.pdf
LPAs/LPA_KKR Global Infrastructure Investors IV (USD) SCSp_PDF.pdf
LPAs/LPA_Oaktree Opportunities Fund X Feeder (Cayman), L.P._fr Simon Poon on 20.03.15 at 5.40 pm(main_13604936_1).PDF
LPAs/LPA_Oaktree Opportunities Fund X, L.P._fr Simon Poon on 20.03.15 at 5.40 pm(main_13604915_1).PDF
LPAs/Main LPA (Amendments on 2nd last page).pdf
LPAs/MIP V - A_R LPA (Executed).P

In [45]:
#s1 = np.array(pdf_names).reshape(155,1)
#s2 = np.array(contents).reshape(155,1)
#s3 = np.array(pagenumendcontents).reshape(155,1)
#data = np.concatenate((s1, s2, s3), axis=1)
#df = pd.DataFrame(data, columns=['pdf_names', 'contents', 'pagenumendcontents'])
#df.to_excel(r'contents.xlsx', index = False, header = True)

In [53]:
df = pd.read_excel('contents.xlsx')

In [1]:
os.listdir()

NameError: name 'os' is not defined